# Taxi Benchmark Analysis

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3, io
from pathlib import Path

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (20, 8)

PLOTS_DIR = Path('plots')
PLOTS_DIR.mkdir(exist_ok=True)

s3 = boto3.client('s3', region_name='eu-north-1')

EXPERIMENT_IDS = [
    "20191006_yellow_Man_LP_MinMaxCostFlow_20251115_221715",
    "20191010_yellow_Man_LP_MinMaxCostFlow_20251116_015121",
    # "20191008_yellow_Man_LP_MinMaxCostFlow_20251116_062435,
]
START_HOUR = 0
TIME_DELTA = 15  # minutes

In [ ]:
# Load data
def load_exp(exp_id):
    resp = s3.list_objects_v2(Bucket='taxi-benchmark', Prefix=f'experiments/{exp_id}/decisions/')
    dfs = []
    for obj in [o for o in resp.get('Contents',[]) if o['Key'].endswith('.parquet')]:
        r = s3.get_object(Bucket='taxi-benchmark', Key=obj['Key'])
        dfs.append(pd.read_parquet(io.BytesIO(r['Body'].read())))
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

experiments = {exp_id: load_exp(exp_id) for exp_id in EXPERIMENT_IDS}
print(f"Loaded {sum(len(df) for df in experiments.values())} records")

In [ ]:
# Helper
def to_hour(tw_idx):
    return START_HOUR + tw_idx * (TIME_DELTA / 60.0)

def agg_hourly(df, field):
    df = df.copy()
    df['hour'] = df['time_window_idx'].apply(to_hour)
    if field == 'opt_value':
        return df.groupby(['hour','time_window_idx'])[field].first().groupby('hour').sum().sort_index()
    return df.groupby('hour')[field].sum().sort_index()

# Define variants
def get_variants(df, method):
    m = df[df['method']==method]
    pl = m[m['acceptance_function']=='PL']
    sig = m[m['acceptance_function']=='Sigmoid']
    return {'PL': pl, 'Sigmoid': sig}

# Colors: Set2
COLORS = {
    'LP_PL': plt.cm.Set2(0),
    'LP_Sigmoid': plt.cm.Set2(1),
    'MinMaxCostFlow_PL': plt.cm.Set2(3),
    'MinMaxCostFlow_Sigmoid': plt.cm.Set2(4),
}

In [ ]:
# Plot Revenue
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,8))

for exp_idx, (exp_id, df) in enumerate(experiments.items()):
    ls = ['-','--','-.', ':'][exp_idx % 4]
    for method in ['LP', 'MinMaxCostFlow']:
        for variant_name, variant_df in get_variants(df, method).items():
            if variant_df.empty: continue
            key = f"{method}_{variant_name}"
            hourly = agg_hourly(variant_df, 'profit').cumsum()
            ax1.plot(hourly.index, hourly.values, color=COLORS[key], linestyle=ls,
                    alpha=0.8, linewidth=2.5, label=f"{key} ({exp_id[:8]})")

ax1.set_xlabel('Hour', fontsize=13)
ax1.set_ylabel('Cumulative Revenue ($)', fontsize=13)
ax1.set_title('Customer Revenue - Individual Experiments', fontsize=14, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 24)

# Averaged
for method in ['LP', 'MinMaxCostFlow']:
    for variant_name in ['PL', 'Sigmoid']:
        key = f"{method}_{variant_name}"
        all_cum = []
        for df in experiments.values():
            variants = get_variants(df, method)
            if variant_name in variants and not variants[variant_name].empty:
                all_cum.append(agg_hourly(variants[variant_name], 'profit').cumsum())
        
        if all_cum:
            hours = sorted(set(h for c in all_cum for h in c.index))
            aligned = pd.DataFrame({f'e{j}': c for j, c in enumerate(all_cum)}, index=hours)
            aligned = aligned.fillna(method='ffill')
            mean, std = aligned.mean(axis=1), aligned.std(axis=1)
            ax2.plot(mean.index, mean.values, color=COLORS[key], linewidth=3,
                    label=key, marker='o', markersize=4)
            ax2.fill_between(mean.index, mean-std, mean+std, color=COLORS[key], alpha=0.15)

ax2.set_xlabel('Hour', fontsize=13)
ax2.set_ylabel('Cumulative Revenue ($)', fontsize=13)
ax2.set_title('Customer Revenue - Averaged', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 24)
plt.tight_layout()
plt.savefig(PLOTS_DIR / '1_revenue.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot Total Value
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,8))

for exp_idx, (exp_id, df) in enumerate(experiments.items()):
    ls = ['-','--','-.', ':'][exp_idx % 4]
    for method in ['LP', 'MinMaxCostFlow']:
        for variant_name, variant_df in get_variants(df, method).items():
            if variant_df.empty: continue
            key = f"{method}_{variant_name}"
            hourly = agg_hourly(variant_df, 'total_value').cumsum()
            ax1.plot(hourly.index, hourly.values, color=COLORS[key], linestyle=ls,
                    alpha=0.8, linewidth=2.5, label=f"{key} ({exp_id[:8]})")

ax1.set_xlabel('Hour', fontsize=13)
ax1.set_ylabel('Cumulative Total Value ($)', fontsize=13)
ax1.set_title('System Total Value - Individual', fontsize=14, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 24)

# Averaged
for method in ['LP', 'MinMaxCostFlow']:
    for variant_name in ['PL', 'Sigmoid']:
        key = f"{method}_{variant_name}"
        all_cum = []
        for df in experiments.values():
            variants = get_variants(df, method)
            if variant_name in variants and not variants[variant_name].empty:
                all_cum.append(agg_hourly(variants[variant_name], 'total_value').cumsum())
        
        if all_cum:
            hours = sorted(set(h for c in all_cum for h in c.index))
            aligned = pd.DataFrame({f'e{j}': c for j, c in enumerate(all_cum)}, index=hours)
            aligned = aligned.fillna(method='ffill')
            mean, std = aligned.mean(axis=1), aligned.std(axis=1)
            ax2.plot(mean.index, mean.values, color=COLORS[key], linewidth=3,
                    label=f"{key} (realized)", marker='o', markersize=4)
            ax2.fill_between(mean.index, mean-std, mean+std, color=COLORS[key], alpha=0.15)

# Add LP optimal
for variant_name in ['PL', 'Sigmoid']:
    key = f"LP_{variant_name}"
    all_opt = []
    for df in experiments.values():
        variants = get_variants(df, 'LP')
        if variant_name in variants and not variants[variant_name].empty:
            all_opt.append(agg_hourly(variants[variant_name], 'opt_value').cumsum())
    
    if all_opt:
        hours = sorted(set(h for o in all_opt for h in o.index))
        aligned = pd.DataFrame({f'e{j}': o for j, o in enumerate(all_opt)}, index=hours)
        aligned = aligned.fillna(method='ffill')
        mean_opt = aligned.mean(axis=1)
        ax2.plot(mean_opt.index, mean_opt.values, '--', color=COLORS[key],
                linewidth=2.5, label=f"{key} (optimal)", marker='s', markersize=3, alpha=0.8)

ax2.set_xlabel('Hour', fontsize=13)
ax2.set_ylabel('Cumulative Total Value ($)', fontsize=13)
ax2.set_title('System Total Value - Averaged (Apple-to-Apple!)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 24)
plt.tight_layout()
plt.savefig(PLOTS_DIR / '2_total_value.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Summary
for exp_id, df in experiments.items():
    print(f"\n{exp_id}:")
    for method in ['LP', 'MinMaxCostFlow']:
        for variant_name, variant_df in get_variants(df, method).items():
            if variant_df.empty: continue
            rev = variant_df['profit'].sum()
            tv = variant_df['total_value'].sum()
            print(f"  {method}_{variant_name}: Revenue=${rev:,.0f}, TotalValue=${tv:,.0f}", end='')
            if method == 'LP':
                opt = variant_df.groupby('time_window_idx')['opt_value'].first().sum()
                print(f", Optimal=${opt:,.0f}, Ratio={tv/opt*100:.1f}%")
            else:
                print()